# Fibonacci sequence with LangGraph
## Reminder :
- F0 = 0  
- F1 = 1 
- Fn = Fn-1 + Fn-2

## Calculation with a State Machine :

```
States (Nodes):
Initial State (Start): The starting state before any calculation begins.
currentState = Start
a = 0
b = 1
counter = 0

while (currentState != End) {
    if (currentState == Start) {
        currentState = Calculate
    } else if (currentState == Calculate) {
        display a
        nextTerm = a + b
        if (counter >= n) {
            currentState = End
        } else {
            currentState = Update
        }
    } else if (currentState == Update) {
        a = b
        b = nextTerm
        counter = counter + 1
        currentState = Calculate
    }
}
```

## State machine implementation with LangGraph :

In [18]:
from devtools import debug  # noqa: F401
from langchain_core.runnables import chain
from langgraph.graph import END, StateGraph
from typing_extensions import TypedDict


class GraphState(TypedDict, total=False):
    a: int
    b: int
    next: int
    counter: int
    sequence: list[int]
    n: int


workflow = StateGraph(GraphState)


@chain
def init(state: GraphState) -> GraphState:
    return state | {"a": 0, "b": 1, "counter": 0, "sequence": []}


@chain
def calculate(state: GraphState) -> GraphState:
    next = state["a"] + state["b"]
    state["next"] = next
    state["sequence"].append(next)
    return state


@chain
def is_end(state: GraphState) -> str:
    if state["counter"] < state["n"]:
        return "update"
    else:
        return "finish"


@chain
def update(state: GraphState) -> GraphState:
    state["a"] = state["b"]
    state["b"] = state["next"]
    state["counter"] = state["counter"] + 1
    return state


# Define the nodes
workflow.add_node("init", init)
workflow.add_node("calculate", calculate)
workflow.add_node("update", update)
workflow.set_entry_point("init")
workflow.add_edge("init", "calculate")
workflow.add_conditional_edges(
    "calculate",
    is_end,
    {
        "update": "update",
        "finish": END,
    },
)
workflow.add_edge("update", "calculate")
app = workflow.compile()

In [19]:
out = app.invoke({"n": 10})
print(out["sequence"])
print(app.get_graph().draw_ascii())

[1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]
        +-----------+          
        | __start__ |          
        +-----------+          
              *                
              *                
              *                
          +------+             
          | init |             
          +------+             
              *                
              *                
              *                
        +-----------+          
        | calculate |          
        +-----------+          
          *        .           
        **          ..         
       *              .        
+--------+        +---------+  
| update |        | __end__ |  
+--------+        +---------+  


### Exercise : 
- Check that 'n' is greater than 1, else print a message.
    - hint: use ```workflow.set_conditional_entry_point(...  ```